### Задание 1

Обязательная часть

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>


In [123]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from datetime import timedelta

In [28]:
KEYWORDS = ['python', 'парсинг', 'писатель', 'Fallout']

In [3]:
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')
posts = soup.find_all(class_='post post_preview')

In [136]:
word_news = pd.DataFrame()
for word in KEYWORDS:
    for post in posts:
        if word in post.find(class_='post__body post__body_crop').get_text():
            date = post.find(class_='post__time').get_text()
            title = post.find(class_='post__title_link').get_text()
            link = post.find(class_='post__title_link').get('href')
            if 'сегодня' in date:
                date = datetime.now().strftime('%d.%m.%Y')
            elif 'вчера' in date:
                date_yest = datetime.now()- timedelta(days=1)
                date = date_yest.strftime('%d.%m.%Y')
            row = {'date': date, 'title': title, 'link': link}
            word_news = pd.concat([word_news, pd.DataFrame([row])])
word_news

,date,title,link
0,20.03.2021,Посмотрим на девочек? Или ml.net в работе,https://habr.com/ru/post/548038/
0,20.03.2021,Сколько стоит UX писатель,https://habr.com/ru/post/547766/
0,19.03.2021,Как IT-комьюнити игроков развивало экосистему ...,https://habr.com/ru/post/548030/


In [6]:
# Черновик
# def check_word(word):
#     word_news = pd.DataFrame()
#     for post in posts:
#         if word in post.find(class_='post__body post__body_crop').get_text():
#             date = post.find(class_='post__time').get_text()
#             title = post.find(class_='post__title_link').get_text()
#             link = post.find(class_='post__title_link').get('href')
#             row = {'date': date, 'title': title, 'link': link}
#             return row
#             word_news = pd.concat([word_news, pd.DataFrame([row])])
#             return word_news

### Задание 2

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.


In [67]:
EMAIL = ["ivan@mail.ru","yasha@ya.ru"]

In [80]:
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
params = {"emailAddresses":EMAIL}
headers = {'Vaar-Version': '0',
'Vaar-Header-App-Build-Version': '1.0.0',
'Vaar-Header-App-Product':'hackcheck-web-avast',
'Vaar-Header-App-Product-Name':'hackcheck-web-avast'
          }

req = requests.post(URL, json = params, headers = headers)

In [50]:
import json

In [82]:
breaches = json.loads(req.text)

In [140]:
breaches_dt = pd.DataFrame()
for mail, breach in breaches['summary'].items():
    for values_breach in breach.values():
        for id_breach in values_breach :
            dt_mail = mail
            id_breachs = str(id_breach) 
            date = breaches['breaches'][id_breachs]['publishDate']
            source = breaches['breaches'][id_breachs]['site']
            description = breaches['breaches'][id_breachs]['description']
            row = {'email': dt_mail, 'date': date[0:9], 'source': source, 'description': description}
            breaches_dt = pd.concat([breaches_dt, pd.DataFrame([row])])
    

In [141]:
breaches_dt

,email,date,source,description
0,ivan@mail.ru,2020-12-0,ta-bao.com,"In November 2020, a collection of over 23,000 ..."
0,ivan@mail.ru,2020-12-0,krym.bazashop.ru,"In November 2020, a collection of over 23,000 ..."
0,ivan@mail.ru,2020-12-1,gorod-59.ru,"In November 2020, a collection of over 23,000 ..."
0,ivan@mail.ru,2019-03-2,bookmate.com,"In July 2018, Bookmate was allegedly breached...."
0,ivan@mail.ru,2019-03-2,roll20.net,"In January 2019, Roll20 allegedly breached. Th..."
...,...,...,...,...
0,ivan@mail.ru,2020-07-1,138gsm.ru,"In February 2019, the Russian phone accessorie..."
0,ivan@mail.ru,2020-12-0,vsarapule.ru,"In November 2020, a collection of over 23,000 ..."
0,ivan@mail.ru,2020-12-1,befutsal.ru,"In November 2020, a collection of over 23,000 ..."
0,ivan@mail.ru,2019-05-2,livejournal.com,"In 2017, social network LiveJournal's database..."
